#8-puzzle

In [1]:
import heapq
import numpy as np
import time
import pandas as pd
import itertools

In [2]:
class Node:
    def __init__(self, state, parent=None, action=None, g=0, h=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.g = g
        self.h = h
        self.f = manhattan_distance(state)+g


    def get_state(self):
        return self.state

    def __lt__(self, other):
        return self.f < other.f

    def __eq__(self, other):
        return np.array_equal(self.state, other.state)

    def __hash__(self):
        return hash(self.state.tostring())

def generate_node(node, action):
    new_state = (node.state).copy()
    zero_position = np.argwhere(new_state == 0)[0]
    move = None
    if action == 'up' and zero_position[0] > 0:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0]-1, zero_position[1]] = \
            new_state[zero_position[0]-1, zero_position[1]], new_state[zero_position[0], zero_position[1]]
        move = (1, 0)
    elif action == 'down' and zero_position[0] < 2:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0]+1, zero_position[1]] = \
            new_state[zero_position[0]+1, zero_position[1]], new_state[zero_position[0], zero_position[1]]
        move = (-1, 0)
    elif action == 'left' and zero_position[1] > 0:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0], zero_position[1]-1] = \
            new_state[zero_position[0], zero_position[1]-1], new_state[zero_position[0], zero_position[1]]
        move = (0, 1)
    elif action == 'right' and zero_position[1] < 2:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0], zero_position[1]+1] = \
            new_state[zero_position[0], zero_position[1]+1], new_state[zero_position[0], zero_position[1]]
        move = (0, -1)
    else:
        return None
    return Node(new_state, node, move, node.g+1, None)

def goal_test(node):
    return np.array_equal(node.state, np.array([[1, 2, 3], [4,5, 6], [ 7, 8,0]]))

def misplaced_tiles(node):
    return np.sum(node.state != np.array([[1, 2, 3], [4,5, 6], [ 7, 8,0]]))

def manhattan_distance(node):
    #goal_state = np.array([[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]])
    goal_state = np.array([[1, 2, 3], [4,5, 6], [ 7, 8,0]])

    if str(type(node)) == '<class \'numpy.ndarray\'>':

        positions = np.array(np.unravel_index(node, (3, 3)), dtype=np.int32)#.state
    else:
        positions = np.array(np.unravel_index(node.state, (3, 3)), dtype=np.int32)#.state
    goal_positions = np.array(np.unravel_index(goal_state, (3, 3)), dtype=np.int32)
    # print(goal_positions)
    return np.sum(np.abs(positions - goal_positions))

def fs(node):
    return node.g + misplaced_tiles(node)

def fu(node):
    if node.f != 0:
        return node.f
    return node.g + manhattan_distance(node)

num_expanded_vertices = 0
def lookahead(node, LHB, UB, MinCost, closed, num_expanded_vertices,flag_close):
    num_expanded_vertices.append(1)
    for action in ['up', 'down', 'left', 'right']:

        child = generate_node(node, action)
        if child is None:
            continue
        if flag_close:
            if child in closed:
                continue

        if goal_test(child):
            UB = fu(child)
            MinCost = min(MinCost, fu(child))
        else:
            if fu(child) >= LHB:
                MinCost = min(MinCost, fu(child))
            else:
                child_min_cost = lookahead(child, LHB, UB, MinCost,closed,num_expanded_vertices,flag_close)
                if child_min_cost is not None:
                    MinCost = min(MinCost, child_min_cost)

    return MinCost

def duplicate_detection(node):
    for n in open:
        if np.array_equal(n.state, node.state):
            return True
    for n in closed:
        if np.array_equal(n.state, node.state):
            return True
    return False

def expansion_cycle_Astar_lookahead(start_state,flag_close,k):
    global open, closed,path
    global num_expanded_vertices
    num_expanded_vertices = []
    UB = np.inf
    LHB = manhattan_distance(start_state)
    min_cost = float('inf')
    open = [Node(start_state, g=0, h=manhattan_distance(start_state))]
    closed = set()
    while open:
        # Pop node with lowest f-value from open list
        v = heapq.heappop(open)
        if v.h == 0:
            return v

        if v.g + manhattan_distance(v) >= UB:
            closed.add(v)
            continue
        for action in ['up', 'down', 'left', 'right']:
            child = generate_node(v, action)
            if child is None:
                continue
            f_u =fu(child)
            if f_u >= UB:
                continue
            if goal_test(child):
                UB = fu(child)

                path = []
                while child.parent is not None:
                    path.append(child.action)
                    child = child.parent
                path.reverse()

            LHB = min(UB, fs(v) + k)

            if fu(child)<=LHB:
                min_cost = lookahead(child, LHB, UB, np.inf,closed,num_expanded_vertices,flag_close)
                if min_cost> child.f:
                    child.f = min_cost
            if not duplicate_detection(child):
                heapq.heappush(open, child)

        closed.add(v)
   
    return None

In [3]:
state_1 = np.array([[1,5,3], [7,4,6], [8,2,0]])
state_2 = np.array([[4,8,6], [3,1,5],[2,7,0]])
state_3 = np.array([[5,3,4], [6,8,2], [1,7,0]])
state_4 = np.array([[7,4,2], [1,3,8], [5,6,0]])
state_5 = np.array([[1,4,7], [5,2,3], [6,8,0]])
state_6 = np.array([[7,8,1],[3,2,6],[5,4,0]])
state_7 = np.array([[2,6,3],[7,8,1],[4,5,0]])
state_8 = np.array([[2,3,6],[4,8,7],[5,1,0]])
state_9 = np.array([[4,8,2],[7,5,6],[1,3,0]])
state_10 = np.array([[5,7,4],[2,8,3],[6,1,0]])
state_11 = np.array([[3,4,1],[5,6,8],[2,7,0]])
state_12 = np.array([[7,8,4],[6,1,5],[3,2,0]])
state_13 = np.array([[1,4,2],[6,3,8],[5,7,0]])
state_14 = np.array([[5,7,6],[3,4,2],[1,8,0]])
state_15 = np.array([[2,7,6],[4,5,1],[3,8,0]])
state_16 = np.array([[1,4,3],[7,8,6],[5,2,0]])
state_17 = np.array([[8,2,1],[6,7,5],[3,4,0]])
state_18 = np.array([[2,8,5],[7,6,3],[1,4,0]])
state_19 = np.array([[1,6,4],[2,3,8],[5,7,0]])
state_20 = np.array([[3,1,2],[7,6,5],[4,8,0]])

state_21 = np.array([[4,1,2],[5,6,7],[3,8,0]])

state_22 = np.array([[7,8,6],[5,2,1],[3,4,0]])

state_23 = np.array([[1,8,7],[3,5,4],[6,2,0]])

state_24 = np.array([[1,2,6],[4,5,8],[7,3,0]])

state_25 = np.array([[7,1,5],[6,2,8],[3,4,0]])

state_26 = np.array([[8,3,1],[7,5,6],[4,2,0]])

state_27 = np.array([[1,2,6],[4,5,8],[7,3,0]])

state_28 = np.array([[4,6,1],[7,5,2],[3,8,0]])

state_29 = np.array([[4,8,7],[2,5,1],[6,3,0]])
state_30 = np.array([[8,1,2],[5,3,7],[6,4,0]])

In [ ]:
# states = [np.array([[2, 3, 5], [1, 4, 6], [7, 8, 0]]),np.array(   [[1, 2, 3],[5, 6, 0],[7, 8, 4]]),np.array([[2, 8, 3], [1, 0, 5], [4, 7, 6]])]
states = [state_1,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15,state_16,state_17,state_18,state_19,state_20,
        state_21,state_22,state_23,state_24,state_25,state_26,state_27,state_28,state_29,state_30 ]
# states = [np.array([[2, 8, 3], [1, 0, 5], [4, 7, 6]])]

data_True = []
data_False = []
k_num = [6,7,8]#,3,4,5
close_list = [True,False]
experiment_params = [states,k_num,close_list]
combinations = list(itertools.product(*experiment_params))

for combination in combinations:
    start_state = combination[0]
    k = combination[1]
    flag_close = combination[2]
    print(start_state)
    start = time.time()
    # Call the A* search algorithm with lookahead to solve the puzzle
    solution_node = expansion_cycle_Astar_lookahead(start_state,flag_close,k)
    end = time.time()

    total_time = end - start
    print("\n"+ str(total_time))

    print("No solution found.")
    print(path)
    print("Number of nodes expanded: ", len(closed))
    print(len(num_expanded_vertices))
    print(flag_close)
    if flag_close:
        data_True.append([start_state,k,total_time,len(closed),len(num_expanded_vertices),flag_close])
    else:
        data_False.append([start_state,k,total_time,len(closed),len(num_expanded_vertices),flag_close])
    
    df_True = pd.DataFrame(data_True,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
    df_False = pd.DataFrame(data_False,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
    df_True.to_csv('df_True6_8.csv')
    df_False.to_csv('df_False6_8.csv')
    
df_True = pd.DataFrame(data_True,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
df_False = pd.DataFrame(data_False,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
df_True.to_csv('df_True_final6_8.csv')
df_False.to_csv('df_False_final6_8.csv')

[[1 5 3]
 [7 4 6]
 [8 2 0]]


/tmp/ipykernel_51283/2632516800.py:21: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return hash(self.state.tostring())



4.96770715713501
No solution found.
[(1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (1, 0), (0, -1), (-1, 0), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (1, 0), (0, -1), (-1, 0), (-1, 0)]
Number of nodes expanded:  1116
2330
True
[[1 5 3]
 [7 4 6]
 [8 2 0]]

8.336483478546143
No solution found.
[(1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (1, 0), (0, -1), (-1, 0), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (1, 0), (0, -1), (-1, 0), (-1, 0)]
Number of nodes expanded:  1285
12177
False
[[1 5 3]
 [7 4 6]
 [8 2 0]]

5.5540571212768555
No solution found.
[(1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (1, 0), (0, -1), (-1, 0), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (1, 0), (0, -1), (-1, 0), (-1, 0)]
Number of nodes expanded:  1117
5034
True
[[1 5 3]
 [7 4 6]
 [8 2 0]]

7.814742565155029
No solution found.
[(1, 0), (0, 1), (-1, 0), (0, 1), (1, 0), (1, 0), (0, -1), (-1, 0), (0, -1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, -1), (1, 0), (0, -1), (-1, 0), (-1, 0)]
Number of nodes expa

15-puzzle

In [2]:
class Node:
    def __init__(self, state, parent=None, action=None, g=0, h=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.g = g
        self.h = h
        self.f = manhattan_distance(state)+g


    def get_state(self):
        return self.state

    def __lt__(self, other):
        return self.f < other.f

    def __eq__(self, other):
        return np.array_equal(self.state, other.state)

    def __hash__(self):
        return hash(self.state.tostring())

def generate_node(node, action):
    new_state = (node.state).copy()
    zero_position = np.argwhere(new_state == 0)[0]
    move = None
    if action == 'up' and zero_position[0] > 0:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0]-1, zero_position[1]] = \
            new_state[zero_position[0]-1, zero_position[1]], new_state[zero_position[0], zero_position[1]]
        move = (1, 0)
    elif action == 'down' and zero_position[0] < 3:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0]+1, zero_position[1]] = \
            new_state[zero_position[0]+1, zero_position[1]], new_state[zero_position[0], zero_position[1]]
        move = (-1, 0)
    elif action == 'left' and zero_position[1] > 0:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0], zero_position[1]-1] = \
            new_state[zero_position[0], zero_position[1]-1], new_state[zero_position[0], zero_position[1]]
        move = (0, 1)
    elif action == 'right' and zero_position[1] < 3:
        new_state[zero_position[0], zero_position[1]], new_state[zero_position[0], zero_position[1]+1] = \
            new_state[zero_position[0], zero_position[1]+1], new_state[zero_position[0], zero_position[1]]
        move = (0, -1)
    else:
        return None
    return Node(new_state, node, move, node.g+1, None)

def goal_test(node):
    return np.array_equal(node.state, np.array([[1, 2, 3,4], [5, 6,7, 8], [ 9, 10,11,12],[ 13, 14,15,0]]))

def misplaced_tiles(node):
    return np.sum(node.state != np.array([[1, 2, 3,4], [5, 6,7, 8], [ 9, 10,11,12],[ 13, 14,15,0]]))

def manhattan_distance(node):
    #goal_state = np.array([[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]])
    goal_state = np.array([[1, 2, 3,4], [5, 6,7, 8], [ 9, 10,11,12],[ 13, 14,15,0]])

    if str(type(node)) == '<class \'numpy.ndarray\'>':

        positions = np.array(np.unravel_index(node, (4, 4)), dtype=np.int32)#.state
    else:
        positions = np.array(np.unravel_index(node.state, (4, 4)), dtype=np.int32)#.state
    goal_positions = np.array(np.unravel_index(goal_state, (4, 4)), dtype=np.int32)
    # print(goal_positions)
    return np.sum(np.abs(positions - goal_positions))

def fs(node):
    return node.g + misplaced_tiles(node)

def fu(node):
    if node.f != 0:
        return node.f
    return node.g + manhattan_distance(node)

num_expanded_vertices = 0
def lookahead(node, LHB, UB, MinCost, closed, num_expanded_vertices,flag_close):
    num_expanded_vertices.append(1)
    for action in ['up', 'down', 'left', 'right']:

        child = generate_node(node, action)
        if child is None:
            continue
        if flag_close:
            if child in closed:
                continue

        if goal_test(child):
            UB = fu(child)
            MinCost = min(MinCost, fu(child))
        else:
            if fu(child) >= LHB:
                MinCost = min(MinCost, fu(child))
            else:
                child_min_cost = lookahead(child, LHB, UB, MinCost,closed,num_expanded_vertices,flag_close)
                if child_min_cost is not None:
                    MinCost = min(MinCost, child_min_cost)

    return MinCost

def duplicate_detection(node):
    for n in open:
        if np.array_equal(n.state, node.state):
            return True
    for n in closed:
        if np.array_equal(n.state, node.state):
            return True
    return False

def expansion_cycle_Astar_lookahead(start_state,flag_close,k):
    global open, closed,path
    global num_expanded_vertices
    num_expanded_vertices = []
    UB = np.inf
    LHB = manhattan_distance(start_state)
    min_cost = float('inf')
    open = [Node(start_state, g=0, h=manhattan_distance(start_state))]
    closed = set()
    while open:
        # Pop node with lowest f-value from open list
        v = heapq.heappop(open)
        if v.h == 0:
            return v

        if v.g + manhattan_distance(v) >= UB:
            closed.add(v)
            continue
        for action in ['up', 'down', 'left', 'right']:
            child = generate_node(v, action)
            if child is None:
                continue
            f_u =fu(child)
            if f_u >= UB:
                continue
            if goal_test(child):
                UB = fu(child)

                path = []
                while child.parent is not None:
                    path.append(child.action)
                    child = child.parent
                path.reverse()

            LHB = min(UB, fs(v) + k)

            if fu(child)<=LHB:
                min_cost = lookahead(child, LHB, UB, np.inf,closed,num_expanded_vertices,flag_close)
                if min_cost> child.f:
                    child.f = min_cost
            if not duplicate_detection(child):
                heapq.heappush(open, child)

        closed.add(v)
   
    return None

In [3]:
state_1 = np.array([[2,6,15,1], [13,5,7,9], [4,3,11,10],[8,14,12,0]])
state_2 = np.array([[10,4,14,12],[9,2,0,3],[8,13,1,6],[11,15,7,5]])
state_3 = np.array([[12,8,10,2],[6,14,1,4],[11,0,9,3],[5,15,7,13]])
state_4 = np.array([[7,5,10,3],[6,14,8,9],[4,1,12,13],[11,0,15,2]])
state_5 = np.array([[15,14,2,4],[0,3,8,10],[13,1,6,11],[5,9,7,12]])
state_6 = np.array([[14,11,7,1],[6,12,15,3],[5,9,2,4],[13,8,0,10]])
state_7 = np.array([[2,0,5,12],[7,9,8,11],[3,4,10,13],[1,14,15,6]])
state_8 = np.array([[2,4,5,15],[3,7,9,6],[14,13,1,0],[12,8,10,11]])
state_9 = np.array([[7,1,6,14],[10,2,12,9],[15,4,5,0],[11,8,3,13]])
state_10 = np.array([[10,15,0,14],[9,13,1,5],[6,3,2,12],[11,4,8,7]])
state_11 = np.array([[13,0,6,4],[9,14,11,12],[10,5,8,1],[2,3,15,7]])
state_12 = np.array([[3,10,1,7],[4,8,2,11],[5,15,13,12],[9,6,14,0]])
state_13 = np.array([[8,9,6,0],[4,5,2,10],[7,1,13,3],[14,15,12,11]])
state_14 = np.array([[15,1,6,4],[11,10,9,7],[2,14,0,13],[12,5,8,3]])
state_15 = np.array([[0,7,1,5],[2,9,10,4],[15,13,6,8],[3,14,12,11]])
state_16 = np.array([[7,6,10,11],[3,1,8,0],[14,15,2,5],[4,13,12,9]])

state_17 = np.array([[6,7,0,1],[9,12,4,3],[15,2,11,14],[13,10,8,5]])
state_18 = np.array([[13,10,11,12],[7,0,2,6],[14,1,15,8],[9,3,5,4]])
state_19 = np.array([[3,1,6,7],[15,8,14,12],[0,11,9,5],[4,13,10,2]])
state_20 = np.array([[13,11,2,5],[9,12,10,1],[14,6,4,3],[0,15,8,7]])
state_21 = np.array([[9,7,3,11],[5,13,2,0],[10,12,15,6],[14,4,8,1]])

state_22 = np.array([[9,11,10,2],[3,6,12,15],[13,14,7,8],[1,4,5,0]])

state_23 = np.array([[9,10,7,0],[13,4,11,12],[1,14,6,8],[5,15,2,3]])

state_24 = np.array([[15,1,12,0],[9,8,2,7],[10,13,14,5],[4,11,6,3]])

state_25 = np.array([[1,10,12,3],[13,2,4,0],[15,5,6,14],[9,7,11,8]])

state_26 = np.array([[9,10,6,15],[11,2,13,0],[1,12,14,8],[5,7,4,3]])

state_27 = np.array([[8,9,6,0],[7,1,13,15],[10,4,3,11],[5,12,14,2]])

state_28 = np.array([[0,9,3,12],[5,11,8,4],[15,13,10,1],[14,6,7,2]])

state_29 = np.array([[0,7,2,13],[9,10,8,15],[3,5,1,12],[6,4,11,14]])
state_30 = np.array([[12,8,5,13],[2,14,9,3],[7,4,0,11],[6,1,15,10]])

In [ ]:
# states = [np.array([[2, 3, 5], [1, 4, 6], [7, 8, 0]]),np.array(   [[1, 2, 3],[5, 6, 0],[7, 8, 4]]),np.array([[2, 8, 3], [1, 0, 5], [4, 7, 6]])]
states = [state_1,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,state_12,state_13,state_14,state_15,state_16,state_17,state_18,state_19,state_20,
        state_21,state_22,state_23,state_24,state_25,state_26,state_27,state_28,state_29,state_30 ]
# states = [np.array([[2, 8, 3], [1, 0, 5], [4, 7, 6]])]

data_True = []
data_False = []
k_num = [13,14,15]#,3,4,5
close_list = [True,False]
experiment_params = [states,k_num,close_list]
combinations = list(itertools.product(*experiment_params))

for combination in combinations:
    start_state = combination[0]
    k = combination[1]
    flag_close = combination[2]
    print(start_state)
    start = time.time()
    # Call the A* search algorithm with lookahead to solve the puzzle
    solution_node = expansion_cycle_Astar_lookahead(start_state,flag_close,k)
    end = time.time()

    total_time = end - start
    print("\n"+ str(total_time))

    print("No solution found.")
    print(path)
    print("Number of nodes expanded: ", len(closed))
    print(len(num_expanded_vertices))
    print(flag_close)
    if flag_close:
        data_True.append([start_state,k,total_time,len(closed),len(num_expanded_vertices),flag_close])
    else:
        data_False.append([start_state,k,total_time,len(closed),len(num_expanded_vertices),flag_close])
    
    df_True = pd.DataFrame(data_True,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
    df_False = pd.DataFrame(data_False,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
    df_True.to_csv('/home/noaai/A_STAR/df_True13_15.csv')
    df_False.to_csv('/home/noaai/df_False13_15.csv')
    
df_True = pd.DataFrame(data_True,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
df_False = pd.DataFrame(data_False,columns=['start_state','k','total_time','expansions','expansions_lookheads','Flag_close_list'])
df_True.to_csv('/home/noaai/df_True_final13_15.csv')
df_False.to_csv('/home/noaai/df_False_final13_15.csv')

[[ 2  6 15  1]
 [13  5  7  9]
 [ 4  3 11 10]
 [ 8 14 12  0]]


/tmp/ipykernel_50178/1870328085.py:21: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return hash(self.state.tostring())
